In [31]:
# dp practice (haven't used Python for a while since working on Julia)
cache = {1:1, 2:1}
def fib(n):
    if n in cache:
        return cache[n]
    current = fib(n-1) + fib(n-2)
    cache[n] = current
    return current

num = 100
print("fib(",num,"):",fib(num))
# print(cache)

# Ahh, it's as great as I remember.
    

fib( 100 ): 354224848179261915075
